In [1]:
from __future__ import print_function
import torch as t
from torch.autograd import Variable as V

In [2]:
x = V(t.ones(2,2),requires_grad = True)
x

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)

In [3]:
y = x.sum()
y

tensor(4., grad_fn=<SumBackward0>)

In [4]:
y.grad_fn

In [5]:
y.backward()
x.grad

tensor([[1., 1.],
        [1., 1.]])

In [6]:
y.backward()
x.grad

tensor([[2., 2.],
        [2., 2.]])

In [7]:
y.backward()
x.grad

tensor([[3., 3.],
        [3., 3.]])

In [8]:
x.grad.data.zero_()
x

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)

In [9]:
#Variable和Tensor接口近乎一致，可以做到无缝切换
x = V(t.ones(4,5))
y = t.cos(x)
x_tensor_cos = t.cos(x.data)
x, y, x.data, x_tensor_cos



(tensor([[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]]),
 tensor([[0.5403, 0.5403, 0.5403, 0.5403, 0.5403],
         [0.5403, 0.5403, 0.5403, 0.5403, 0.5403],
         [0.5403, 0.5403, 0.5403, 0.5403, 0.5403],
         [0.5403, 0.5403, 0.5403, 0.5403, 0.5403]]),
 tensor([[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]]),
 tensor([[0.5403, 0.5403, 0.5403, 0.5403, 0.5403],
         [0.5403, 0.5403, 0.5403, 0.5403, 0.5403],
         [0.5403, 0.5403, 0.5403, 0.5403, 0.5403],
         [0.5403, 0.5403, 0.5403, 0.5403, 0.5403]]))

In [10]:
a = V(t.ones(3,4), requires_grad = True)
b = V(t.zeros(3,4))
c = a + b # 等价于 a.add(b)
print(a)
print(b)
print(c)

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], requires_grad=True)
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], grad_fn=<AddBackward0>)


### autograd计算的导数 VS 手动推导的导数
## $$y=x^2e^x$$
### 导数为：
## $$2xe^x+x^2e^x$$

In [11]:
def f(x):
    #原式
    y = x ** 2 * t.exp(x)
    return y
    
    #手动求导方法
def gradf(x):
    dx = 2 * x * t.exp(x) + x ** 2 * t.exp(x)
    return dx

In [12]:
#手动推导
x = V(t.randn(3,4), requires_grad = True)
y = f(x)
dx = gradf(x)
print(y)
print(dx)

tensor([[1.2099e-01, 2.4156e-02, 9.3368e-01, 5.2788e-01],
        [3.5220e-02, 3.0246e-01, 3.3324e-01, 5.9175e+00],
        [5.2048e-01, 5.6312e-05, 5.9863e-01, 1.1042e+00]],
       grad_fn=<MulBackward0>)
tensor([[-4.3965e-01,  3.5830e-01,  3.6567e+00, -9.3452e-02],
        [ 4.4431e-01, -4.2210e-01, -3.9933e-01,  1.5152e+01],
        [-1.1844e-01, -1.4896e-02,  2.6658e+00,  4.1311e+00]],
       grad_fn=<AddBackward0>)


In [13]:
#autograd
y.backward(t.ones(3, 4))
print(x.grad)

tensor([[-4.3965e-01,  3.5830e-01,  3.6567e+00, -9.3452e-02],
        [ 4.4431e-01, -4.2210e-01, -3.9933e-01,  1.5152e+01],
        [-1.1844e-01, -1.4896e-02,  2.6658e+00,  4.1311e+00]])


### 计算图(叶变量、根变量)

In [14]:
x = V(t.ones(1))
w = V(t.randn(1), requires_grad=True)
b = V(t.randn(1), requires_grad=True)
y = w * x # 等价于 w.mul(x)
z = y + b 

In [15]:
x.requires_grad, w.requires_grad, b.requires_grad

(False, True, True)

In [16]:
x.is_leaf, w.is_leaf, b.is_leaf

(True, True, True)

In [17]:
y.requires_grad, z.requires_grad

(True, True)

In [18]:
y.is_leaf, z.is_leaf

(False, False)

In [19]:
z.grad_fn

In [20]:
x = V(t.ones(1), requires_grad=True)
w = V(t.rand(1), requires_grad=True)
y = w * x
z = y.sum()
x.requires_grad, y.requires_grad, w.requires_grad, x, y, w

(True,
 True,
 True,
 tensor([1.], requires_grad=True),
 tensor([0.9698], grad_fn=<MulBackward0>),
 tensor([0.9698], requires_grad=True))

In [21]:
#z.backward()
(x.grad, y.grad, w.grad)

(None, None, None)

In [22]:
#对比上方结果，与x,y,w
z.backward()    #从z开始反向传播，也可以换y.backward(),从y开始反向传播
(x.grad, y.grad, w.grad)

(tensor([0.9698]), None, tensor([1.]))

In [23]:
x = V(t.rand(3), requires_grad=True)
w = V(t.rand(3), requires_grad=True)
y = w * x
z = y.sum()
print(x, w)
print(t.autograd.grad(z, y)) #z对y的梯度
print(t.autograd.grad(z, x)) #z对x的梯度

tensor([0.9074, 0.4474, 0.6749], requires_grad=True) tensor([0.9425, 0.9263, 0.1340], requires_grad=True)
(tensor([1., 1., 1.]),)
(tensor([0.9425, 0.9263, 0.1340]),)
